In [45]:
# Define the neural network architecture using PyTorch as usual

import torch
import torch.nn as nn

class ReLUConvBN(nn.Module):
    def __init__(self, out_channels, kernel_size, stride, padding):
        super().__init__()

        self.kernel_size = kernel_size
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.LazyConv2d(
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                dilation=2,
                bias=False,
            ),
            nn.LazyBatchNorm2d(affine=True, track_running_stats=True),
        )

    def forward(self, x):
        return self.op(x)


class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x
    


In [46]:
# Define the NEPS space for the neural network architecture

from neps.space.neps_spaces.parameters import Pipeline, Operation, Categorical, Resampled

class NN_Space(Pipeline):
    _id = Operation(operator=Identity)
    _three = Operation(operator=nn.Conv2d,kwargs={"in_channels":3, "out_channels":3, "kernel_size":3, "stride":1, "padding":1})
    _one = Operation(operator=nn.Conv2d,kwargs={"in_channels":3, "out_channels":3, "kernel_size":1, "stride":1, "padding":0})
    _reluconvbn = Operation(operator=ReLUConvBN, kwargs={"out_channels":3, "kernel_size":3, "stride":1, "padding":1})

    _O = Categorical(choices=(_three, _one, _id))

    _C_ARGS = Categorical(
        choices=(
            (Resampled(_O),),
            (Resampled(_O), Resampled("model"), _reluconvbn),
            (Resampled(_O), Resampled("model")),
            (Resampled("model"),),
        ),
    )
    _C = Operation(
        operator=nn.Sequential,
        args=Resampled(_C_ARGS),
    )

    _model_ARGS = Categorical(
        choices=(
            (Resampled(_C),),
            (_reluconvbn,),
            (Resampled("model"),),
            (Resampled("model"), Resampled(_C)),
            (Resampled(_O), Resampled(_O), Resampled(_O)),
            (
                Resampled("model"),
                Resampled("model"),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
                Resampled(_O),
            ),
        ),
    )
    model = Operation(
        operator=nn.Sequential,
        args=Resampled(_model_ARGS),
    )

In [47]:
# Sampling and printing one random configuration of the pipeline

from neps.space.neps_spaces import neps_space

pipeline = NN_Space()
resolved_pipeline, resolution_context = neps_space.resolve(pipeline)

s = resolved_pipeline.model
s_config_string = neps_space.convert_operation_to_string(s)
pretty_config = neps_space.config_string.ConfigString(s_config_string).pretty_format()
s_callable = neps_space.convert_operation_to_callable(s)

print("Callable:\n")
print(s_callable)

print("\n\nConfig string:\n")
print(pretty_config)

Callable:

Sequential(
  (0): ReLUConvBN(
    (op): Sequential(
      (0): ReLU()
      (1): LazyConv2d(0, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2), bias=False)
      (2): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)


Config string:

(<class 'torch.nn.modules.container.Sequential'> (<class '__main__.ReLUConvBN'> {'out_channels': 3, 'kernel_size': 3, 'stride': 1, 'padding': 1}))
	01 :: <class 'torch.nn.modules.container.Sequential'>
		02 :: <class '__main__.ReLUConvBN'> {'out_channels': 3, 'kernel_size': 3, 'stride': 1, 'padding': 1}


In [48]:
# Defining the pipeline, using the model from the NN_space space as callable

import numpy as np

def evaluate_pipeline(model: nn.Sequential):
    x = torch.ones(size=[1, 3, 220, 220])
    result = np.sum(model(x).detach().numpy().flatten())
    return result

In [49]:
from neps.space.neps_spaces.optimizers.algorithms import RandomSearch
import neps

pipeline_space = NN_Space()

neps.run(
    evaluate_pipeline=evaluate_pipeline,
    pipeline_space=pipeline_space,
    optimizer=RandomSearch,
    root_directory="results/neps_spaces_nn_example",
    post_run_summary=True,
    max_evaluations_total=5,
    overwrite_working_directory=True,
)
neps.status("results/neps_spaces_nn_example", print_summary=True)

# Configs: 5

    success: 5

# Best Found (config 3):

    objective_to_minimize: -56006.5703125
    config: {'SAMPLING__Resolvable.model.args.resampled_categorical::categorical__6': 4, 'SAMPLING__Resolvable.model.args[0].resampled_categorical::categorical__3': 2, 'SAMPLING__Resolvable.model.args[1].resampled_categorical::categorical__3': 1, 'SAMPLING__Resolvable.model.args[2].resampled_categorical::categorical__3': 2}
    path: C:\Users\Amega\Git\neps\neps_examples\neps_spaces\results\neps_spaces_nn_example\configs\config_3


(    config.SAMPLING__Resolvable.model.args.resampled_categorical::categorical__6  \
 id                                                                                 
 1                                                   2                              
 2                                                   2                              
 3                                                   4                              
 4                                                   4                              
 5                                                   4                              
 
     config.SAMPLING__Resolvable.model.args[0].resampled_operation.args.resampled_categorical::categorical__6  \
 id                                                                                                             
 1                                                   1                                                          
 2                                                   2          